In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = 'IlyaGusev/saiga_llama3_8b'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    # load_in_8bit=True,
    torch_dtype = torch.bfloat16,
    device_map = "auto"
)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head)

In [3]:
from transformers import GenerationConfig
generation_config = GenerationConfig.from_pretrained(model_id)
print(generation_config)

GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": 128009,
  "max_new_tokens": 1536,
  "pad_token_id": 128000,
  "repetition_penalty": 1.12,
  "temperature": 0.2,
  "top_k": 30,
  "top_p": 0.9
}



In [4]:
type(generation_config)

transformers.generation.configuration_utils.GenerationConfig

In [5]:
generation_config.max_new_tokens = 300
generation_config.temperature = 0.17
generation_config.top_k= 20
generation_config.top_p= 0.8

In [6]:
def get_llm_answer(query, chunks_join):
    user_prompt = '''Используй только следующий контекст, чтобы очень кратко ответить на вопрос в конце.
    Не пытайся выдумывать ответ.
    Контекст:
    ===========
    {chunks_join}
    ===========
    Вопрос:
    ===========
    {query}'''.format(chunks_join=chunks_join, query=query)
    
    SYSTEM_PROMPT = "Ты система поддержки пользователей компании Росатом. Ты разговариваешь с людьми и помогаешь им."
    RESPONSE_TEMPLATE = "<|im_start|>assistant\n"
    
    prompt = f'''<|im_start|>system\n{SYSTEM_PROMPT}<|im_end|>\n<|im_start|>user\n{user_prompt}<|im_end|>\n{RESPONSE_TEMPLATE}'''
    
    def generate(model, tokenizer, prompt):
        data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
        data = {k: v.to(model.device) for k, v in data.items()}
        output_ids = model.generate(
            **data,
            generation_config=generation_config
        )[0]
        output_ids = output_ids[len(data["input_ids"][0]) :]
        output = tokenizer.decode(output_ids, skip_special_tokens=True)
        return output.strip()
    
    response = generate(model, tokenizer, prompt)
    
    return response

## inference

In [7]:
import gradio as gr

In [8]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

In [9]:
# embeddings = HuggingFaceEmbeddings(model_name="cointegrated/LaBSE-en-ru")

In [10]:
# from utils.reranker import real_rerank
from utils.vector_data import search_best_from_structured, search_best_from_unstructured



/home/yagor/anaconda3/envs/torch/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
query = 'Как и куда загружать выписки по филиалу акционерного общества «РУСБУРМАШ» в Республике Казахстан в системе XС ERP X.X?'

In [12]:
structured_res = search_best_from_structured(query)
structured_res

Как и куда загружать выписки по филиалу акционерного общества «РУСБУРМАШ» в Республике Казахстан в системе XС ERP X.X?


Document(page_content='Добрый день. Подскажите пожалуйста,  где сейчас в системе XС ERP X.X найти счета-фактуры выданные на аванс ?', metadata={'Код': 'IM22439872', 'Категория': 'запрос на обслуживание', 'Время регистрации': '16.01.2024 11:03:21', 'Рабочая группа': '1с erp 2.0 1-я линия', 'Краткое описание': 'Счет-фактура на аванс', 'Решение': 'Добрый день Оксана Владимировна! Перейти к счетам-фактурам выданным на аванс можно так: Финансовый результат и контроллинг – НДС – Счета-фактуры и таможенные документы. Здесь нажать на ссылку «Настроить», в открывшемся окне нажать «Снять пометки», отметить галочкой только «Счет-фактура выданный (аванс)», нажать «Выбрать». Далее можно сделать отбор по организации и/или контрагенту, а также задать необходимый период. Настроенный журнал можно сохранить в избранном, нажав на звездочку.', 'Аналитика 1': 'НДС', 'Аналитика 2': 'Учет НДС', 'Аналитика 3': 'Счета-фактуры'})

In [13]:
structured_res

Document(page_content='Добрый день. Подскажите пожалуйста,  где сейчас в системе XС ERP X.X найти счета-фактуры выданные на аванс ?', metadata={'Код': 'IM22439872', 'Категория': 'запрос на обслуживание', 'Время регистрации': '16.01.2024 11:03:21', 'Рабочая группа': '1с erp 2.0 1-я линия', 'Краткое описание': 'Счет-фактура на аванс', 'Решение': 'Добрый день Оксана Владимировна! Перейти к счетам-фактурам выданным на аванс можно так: Финансовый результат и контроллинг – НДС – Счета-фактуры и таможенные документы. Здесь нажать на ссылку «Настроить», в открывшемся окне нажать «Снять пометки», отметить галочкой только «Счет-фактура выданный (аванс)», нажать «Выбрать». Далее можно сделать отбор по организации и/или контрагенту, а также задать необходимый период. Настроенный журнал можно сохранить в избранном, нажав на звездочку.', 'Аналитика 1': 'НДС', 'Аналитика 2': 'Учет НДС', 'Аналитика 3': 'Счета-фактуры'})

In [14]:
unstructured_res = search_best_from_unstructured(query)
unstructured_res

Document(page_content='Автор  АО «Гринатом»  Инструкция пользователей « D-1C1-1.10. 19 \nФормирование налоговой декларации по \nналогу на прибыль » Страница  71 \nСтраниц  83 \nВерсия  5 \n \n1.4.7 «Склейка» налоговой декларации  \nДля сохранения целостности данных при формировании отчетности в \nпереходный период – с 1C:ERP Росатом на ERP 2.0 реализован функционал, \nкоторый позволяет:  \n\uf02d выгрузить сформированную в 1С:  ERP (УПП) декларацию \n(Прибыль (внешний)) в файл .xml;  \n\uf02d загрузить файл с отчетами в ERP 2.0, используя функционал \nзагрузки отчетов;  \n\uf02d добавить в новый отчет данные за отсутствующий период этого \nгода или предыдущие требуемые периоды.  \n \nВ пакете «Налоговая отчетность» – «Прибыль (внешний)» сформировать \nнужный для выгрузки отчет – в ед. измерения «в рублях», с точностью = 0  \n(Рисунок  69). \n \nРисунок  69: Декларация по налогу на прибыль  \n \nПоследовательность действий:  \n1. Выгрузить  «Декларации по налогу на прибыль» из системы 1

In [15]:
import streamlit as st
from utils import streamlit_texts as TEXTS, system_prompt, prompt_postfix
import logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger(__name__)

st.markdown(TEXTS.SIDEBAR_STYLE, unsafe_allow_html=True)

with st.sidebar:
    # st.image(os.path.join("resources", "img", "logo.jpeg"))
    st.markdown(TEXTS.COMMAND_EXAMPLES)
    st.code(TEXTS.ECONOMY_DECREASE)
    st.code(TEXTS.LOAN_VOLUME)
    st.code(TEXTS.LTIP)
    st.code(TEXTS.DEFAULT)
    st.code(TEXTS.RENT_PAYMENTS)
    # st.markdown(TEXTS.EXAMPLE_PAPER)

if "messages" not in st.session_state:
    st.session_state.messages = [{"role": "assistant", "content": "Привет, я ИИ Ассистент!"}]

if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

if "metadata" not in st.session_state:
    st.session_state.metadata = None

for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"], unsafe_allow_html=True)

if prompt := st.chat_input("Обратитесь ко мне..."):
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt, unsafe_allow_html=True)

    with st.spinner(TEXTS.WAITING):
        with st.chat_message("assistant"):
            # Запуск агента
            logger.info(str(st.session_state.chat_history))
            try:
                result = agent_executor.invoke(
                    {
                        "chat_history": st.session_state.chat_history,
                        "input": prompt + prompt_postfix,
                    }
                )
            except Exception as e:
                logger.error(str(e))
                result = {"output": TEXTS.SORRY}

            image=None
            # Handling complex outputs
            if type(result["output"]) == dict:
                response = result["output"]["result"]
                if "image" in result["output"].keys():
                    image = result["output"]["image"]
                # st.session_state.metadata = result["output"]["metadata"]
                # st.session_state.chat_history.append(HumanMessage(content=str(st.session_state.metadata)))
            else:
                response = result["output"]

            # обновление истории диалога
            # st.session_state.chat_history.append(HumanMessage(content=prompt))
            # st.session_state.chat_history += result["output"]["intermediate_steps"]
            # st.session_state.chat_history.append(AIMessage(content=response))
            # don't vanish the system prompt

            # рисуем ответ
            st.markdown(response, unsafe_allow_html=True)      
            # if image:
            #     st.image(result["output"]["image"])

    st.session_state.messages.append({"role": "assistant", "content": response})

ImportError: cannot import name 'streamlit_texts' from 'utils' (/home/yagor/Рабочий стол/atomic/AtomHack/app/utils/__init__.py)

In [ ]:

history = []
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    button = gr.Button("Отправить")
    clear = gr.ClearButton([msg, chatbot])
    def respond(input, history=None):
        history = []
        res = ''
        excel_chunk = search_best_from_structured(input)
        if excel_chunk:
            print(excel_chunk)
            res += f'Решение аналогичного вопроса: {excel_chunk.metadata["Решение"]}'
            category = ' | '.join([f'{i}: {str(excel_chunk.metadata[i])}' for i in ["Аналитика 1", "Аналитика 2", "Аналитика 3"] ])
            res += f'\n {category}'
        
        pdf_chunk = search_best_from_unstructured(input)
        result = get_llm_answer(query, pdf_chunk.page_content).split('|im_end|>')[0]
        page = pdf_chunk.metadata["source"].split("/")[-1].split("_")[-1].split(".")[0]
        file_name = pdf_chunk.metadata['source'].split('/')[-2] + ".pdf"
        info = f'Страница: {page}, Исходный документ: {file_name}'
        res += '\n' + f'Ответ LLM: {result}' + '\n' + info
        if history is None:
            history = []
        history.append((input, res))
        return '', history

    # def respond(message, chat_history):
    #     bot_message = random.choice(["How are you?", "I love you", "I'm very hungry"])
    #     chat_history.append((message, bot_message))
    #     return "", chat_history
    
    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    button.click(respond, [msg, chatbot], [msg, chatbot])
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7869


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Running on public URL: https://5e952f9840b92954c8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
history

[]

In [16]:
import pandas as pd

In [17]:
questions = pd.read_excel('/home/yagor/Рабочий стол/atomic/AtomHack/data/test_data/test_dataset/submission.xlsx', index_col=0)

In [18]:
questions

,Вопрос,Имя файла с инструкцией,Раздел в инструкции,Страница,Ответ
Индекс,,,,,
0,Как отразить план работы ТС,NaN,NaN,NaN,NaN
1,Как сгенерировать платежное поручение,NaN,NaN,NaN,NaN
2,Как выполнить переоценку валютных договоров ДЦИ,NaN,NaN,NaN,NaN
3,Не могу выполнить проверку сделки в ЕОСДО,NaN,NaN,NaN,NaN
4,Варианты настройки действий при превышении рез...,NaN,NaN,NaN,NaN
5,Как отразить возврат денег в кассу,NaN,NaN,NaN,NaN
6,"Как подписать документ приемка ТМЦ, если нет с...",NaN,NaN,NaN,NaN
7,Какие отчеты предназначены для детального анал...,NaN,NaN,NaN,NaN
8,Что отражает колонка Встречная задолженность в...,NaN,NaN,NaN,NaN


In [19]:
for i in range(len(questions)):
    temp_row = questions.iloc[i, :]
    query = temp_row['Вопрос']
    res = ''
    pdf_chunk = search_best_from_unstructured(query)
    result = get_llm_answer(query, pdf_chunk.page_content).split('|im_end|>')[0]
    questions.loc[i, 'Ответ'] = result
    page = pdf_chunk.metadata["source"].split("/")[-1].split("_")[-1].split(".")[0]
    questions.loc[i, 'Страница'] = page
    file_name = pdf_chunk.metadata['source'].split('/')[-2] + ".pdf"
    questions.loc[i, 'Имя файла с инструкцией'] = file_name
    # new_dict['Файл'].append(file_name)
    # info = f'Страница: {page}, Исходный документ: {file_name}'
    # res += '\n ' + f'Ответ LLM: {result}' + '\n' + info
    # new_dict['Мой полный ответ'].append(res)

In [20]:
questions

,Вопрос,Имя файла с инструкцией,Раздел в инструкции,Страница,Ответ
Индекс,,,,,
0,Как отразить план работы ТС,D-1C1-1.05.01 Планирование работы ТС.pdf,NaN,8,Отразить план работы ТС можно в виде составной...
1,Как сгенерировать платежное поручение,Инструкция_D_1C1_1_10_41_Учет_операций_при_зак...,NaN,15,Сгенерируйте платежное поручение через систему...
2,Как выполнить переоценку валютных договоров ДЦИ,Инструкция_D_1C1_1_04_12_Учет_договоров_длител...,NaN,9,Для выполнения переоценки валютных договоров Д...
3,Не могу выполнить проверку сделки в ЕОСДО,Инструкция_D_1C1_1_01_01_Универсальное_согласо...,NaN,31,"Привет! Я Сайга, твой автоматический помощник...."
4,Варианты настройки действий при превышении рез...,Инструкция_администратора_Описание_настроек_по...,NaN,18,В системе для настройки действий при превышени...
5,Как отразить возврат денег в кассу,Инструкция_D_1C1_1_10_04_Учет_банковских_опера...,NaN,125,Отразить возврат денег в кассе можно через дог...
6,"Как подписать документ приемка ТМЦ, если нет с...",Инструкция_D_1C1_1_19_01_Поступление_ТМЦ_на_ск...,NaN,19,Для подписания документа приемки ТМЦ без серти...
7,Какие отчеты предназначены для детального анал...,Инструкция_D_1C1_1_10_41_Учет_операций_при_зак...,NaN,98,Для детального анализа книг покупок и продаж и...
8,Что отражает колонка Встречная задолженность в...,Инструкция_D_1C1_1_10_39_Сверка_взаиморасчетов...,NaN,18,"Колонка ""Встречная задолженность"" в документе ..."


In [21]:
questions.to_excel('submission_hacken4221.xlsx')

In [22]:
questions = pd.read_excel('/home/yagor/Рабочий стол/atomic/AtomHack/submission_hacken4221.xlsx', index_col=0)

In [23]:
questions

,Вопрос,Имя файла с инструкцией,Раздел в инструкции,Страница,Ответ
Индекс,,,,,
0,Как отразить план работы ТС,D-1C1-1.05.01 Планирование работы ТС.pdf,NaN,8,Отразить план работы ТС можно в виде составной...
1,Как сгенерировать платежное поручение,Инструкция_D_1C1_1_10_41_Учет_операций_при_зак...,NaN,15,Сгенерируйте платежное поручение через систему...
2,Как выполнить переоценку валютных договоров ДЦИ,Инструкция_D_1C1_1_04_12_Учет_договоров_длител...,NaN,9,Для выполнения переоценки валютных договоров Д...
3,Не могу выполнить проверку сделки в ЕОСДО,Инструкция_D_1C1_1_01_01_Универсальное_согласо...,NaN,31,"Привет! Я Сайга, твой автоматический помощник...."
4,Варианты настройки действий при превышении рез...,Инструкция_администратора_Описание_настроек_по...,NaN,18,В системе для настройки действий при превышени...
5,Как отразить возврат денег в кассу,Инструкция_D_1C1_1_10_04_Учет_банковских_опера...,NaN,125,Отразить возврат денег в кассе можно через дог...
6,"Как подписать документ приемка ТМЦ, если нет с...",Инструкция_D_1C1_1_19_01_Поступление_ТМЦ_на_ск...,NaN,19,Для подписания документа приемки ТМЦ без серти...
7,Какие отчеты предназначены для детального анал...,Инструкция_D_1C1_1_10_41_Учет_операций_при_зак...,NaN,98,Для детального анализа книг покупок и продаж и...
8,Что отражает колонка Встречная задолженность в...,Инструкция_D_1C1_1_10_39_Сверка_взаиморасчетов...,NaN,18,"Колонка ""Встречная задолженность"" в документе ..."


In [ ]:
new_dict = {'Вопрос', 'Имя файла с инструкцией': [], 'Раздел в инструкции': [], 'Страница':[], 'Ответ':[]}

In [87]:
new_dict = {i: [] for i in questions.columns if i not in ['Категория']} | {'Мой полный ответ': [], 'Мой ответ': []}

In [90]:
new_dict

{'Вопрос': [],
 'Правильный ответ': [],
 'Файл': [],
 'Страница': [],
 'Мой полный ответ': [],
 'Мой ответ': []}

In [ ]:
for i in range(len(questions)):
    temp_row = questions.iloc[i, :]

In [91]:
for i in range(len(questions)):
    temp_row = questions.iloc[i, :]
    new_dict['Вопрос'].append(temp_row['Вопрос'])
    res = ''
    pdf_chunk = search_best_from_unstructured(temp_row['Вопрос'])
    result = get_llm_answer(temp_row['Вопрос'], pdf_chunk.page_content).split('|im_end|>')[0]
    new_dict['Мой ответ'].append(result)
    new_dict['Правильный ответ'].append(result)
    page = pdf_chunk.metadata["source"].split("/")[-1].split("_")[-1].split(".")[0]
    new_dict['Страница'].append(page)
    file_name = pdf_chunk.metadata['source'].split('/')[-2] + ".pdf"
    new_dict['Файл'].append(file_name)
    info = f'Страница: {page}, Исходный документ: {file_name}'
    res += '\n ' + f'Ответ LLM: {result}' + '\n' + info
    new_dict['Мой полный ответ'].append(res)

In [92]:
new_dict

{'Вопрос': ['Кто такие дополнительные согласующие?',
  'Что делает регламентное задание «Генерация платежных поручений (Вечер)»?',
  'Что делает флаг «Облагается НДС у покупателя товара»?',
  'Что делает кнопка «Создать документы списания»?',
  'Как отразить реализацию без перехода права собственности?',
  'Как создать Дополнительное соглашение к договору?',
  'Как оформить поступление денежных документов от поставщика?',
  'Какие данные отображаются на рабочем месте по работе с денежными документами?',
  'Какие документы входят и исходят при формировании документа «Поступление денежных документов»?',
  'Почему документ Взаимозачет задолженности может быть заблокирован?',
  'Какие действия необходимы для формирования документа выдачи денежных документов?',
  'Что показывае отчет «Задолженность клиентов по срокам»?'],
 'Правильный ответ': ['Дополнительные согласующие — это пользователи, которые, помимо основного согласующего, также участвуют в процессе согласования текущего этапа. Они д

In [98]:
print('\n yy')


 yy


In [66]:
del new_dict['Категория']

In [94]:
df = pd.DataFrame(new_dict)

In [97]:
df

,Вопрос,Правильный ответ,Файл,Страница,Мой полный ответ,Мой ответ
0,Кто такие дополнительные согласующие?,"Дополнительные согласующие — это пользователи,...",Инструкция_D_1C1_1_01_01_Универсальное_согласо...,22,\nОтвет LLM: Дополнительные согласующие — это ...,"Дополнительные согласующие — это пользователи,..."
1,Что делает регламентное задание «Генерация пла...,Регламентное задание «Генерация платежных пору...,Инструкция_D_1C1_1_01_09_Формирование_платежны...,7,\nОтвет LLM: Регламентное задание «Генерация п...,Регламентное задание «Генерация платежных пору...
2,Что делает флаг «Облагается НДС у покупателя т...,"Флаг ""Облагается НДС у покупателя товара"" указ...",Инструкция D-1C1-1.10.14 Учет НДС_v7.pdf,26,"\nОтвет LLM: Флаг ""Облагается НДС у покупателя...","Флаг ""Облагается НДС у покупателя товара"" указ..."
3,Что делает кнопка «Создать документы списания»?,Кнопка «Создать документы списания» использует...,D_1C1_1_18_03_Реализация_продукции_товаров_раб...,31,\nОтвет LLM: Кнопка «Создать документы списани...,Кнопка «Создать документы списания» использует...
4,Как отразить реализацию без перехода права соб...,Отразить реализацию без перехода права собстве...,D_1C1_1_18_03_Реализация_продукции_товаров_раб...,4,\nОтвет LLM: Отразить реализацию без перехода ...,Отразить реализацию без перехода права собстве...
5,Как создать Дополнительное соглашение к договору?,Для создания Дополнительного соглашения к дого...,Инструкция_D_1C1_1_13_01_Ведение_договоров_на_...,19,\nОтвет LLM: Для создания Дополнительного согл...,Для создания Дополнительного соглашения к дого...
6,Как оформить поступление денежных документов о...,Для оформления поступления денежных документов...,Инструкция_пользователей_«D_1C1_1_22_01_Формир...,33,\nОтвет LLM: Для оформления поступления денежн...,Для оформления поступления денежных документов...
7,Какие данные отображаются на рабочем месте по ...,На рабочем месте по работе с денежными докумен...,Инструкция_D_1C1_1_10_02_Учет_денежных_докумен...,7,\nОтвет LLM: На рабочем месте по работе с дене...,На рабочем месте по работе с денежными докумен...
8,Какие документы входят и исходят при формирова...,Входят: \n- Приходная накладная\n- Банковская ...,Инструкция_D_1C1_1_10_02_Учет_денежных_докумен...,15,\nОтвет LLM: Входят: \n- Приходная накладная\n...,Входят: \n- Приходная накладная\n- Банковская ...
9,Почему документ Взаимозачет задолженности може...,"Документ ""Взаимозачет задолженности"" может быт...",Инструкция_D_1C1_1_10_39_Сверка_взаиморасчетов...,36,"\nОтвет LLM: Документ ""Взаимозачет задолженнос...","Документ ""Взаимозачет задолженности"" может быт..."


In [20]:
import pandas as pd

In [21]:
df = pd.read_csv('/home/yagor/Рабочий стол/atomic/AtomHack/data/q_by_pdf/ответы на вопросы.csv')

In [22]:
for i in range(len(df)):
    temp_row = df.iloc[i, :]
    print('Вопрос:',	temp_row['Вопрос'])
    print('Правильный ответ:',	temp_row['Правильный ответ'])
    print('Мой ответ:',	temp_row['Мой ответ'])
    print('----------------------------------------------------------------')

Вопрос: Кто такие дополнительные согласующие?
Правильный ответ: Дополнительные согласующие – это пользователи, которые наряду с основным согласующим должны выполнить согласование текущего этапа. Этап будет пройден, когда его согласуют сначала все дополнительные согласующие, а затем основной согласующий.
Мой ответ: Дополнительные согласующие — это пользователи, которые, кроме основного согласующего, также должны выполнить согласование текущего этапа. Они участвуют в процессе согласования до тех пор, пока их согласие не получено, после чего начинается согласование основным согласующим. <
----------------------------------------------------------------
Вопрос: Что делает регламентное задание «Генерация платежных поручений (Вечер)»?
Правильный ответ: Регламентное задание «Генерация платежных поручений (Вечер)» (Новое регламентное задание) создает документы списания денежных средств на основании утвержденных заявок, дата платежа в которых меньше или равно текущей либо больше текущей.
Мой от

In [ ]:
import os

In [99]:
df.to_csv(os.path.join('..', 'data', 'q_by_pdf', 'ответы на вопросы.csv'))

In [3]:
df

NameError: name 'df' is not defined

In [ ]:
QnA_with_LLM('/home/yagor/Рабочий стол/atomic/AtomHack/data/q_by_pdf/вопросы.xlsx')

In [29]:
input

<bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x7e0b96f316a0>>

In [30]:
!pip install fastapi nest-asyncio pyngrok uvicorn

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [24]:
!ngrok config add-authtoken 2Wtb0tJLYUY3SgsEWIvtX7TUKBg_4S2cRAbKoZjNZ22z9Rqj1

Authtoken saved to configuration file: /home/yagor/.config/ngrok/ngrok.yml


In [25]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

# middlewares
app.add_middleware(
    CORSMiddleware, # https://fastapi.tiangolo.com/tutorial/cors/
    allow_origins=['*'], # wildcard to allow all, more here - https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Access-Control-Allow-Origin
    allow_credentials=True, # https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Access-Control-Allow-Credentials
    allow_methods=['*'], # https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Access-Control-Allow-Methods
    allow_headers=['*'], # https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Access-Control-Allow-Headers
)

@app.get('/')
async def root():
    return {'hello': 'world'}

In [27]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn


# specify a port
port = 8000
ngrok_tunnel = ngrok.connect(port)

# where we can visit our fastAPI app
print('Public URL:', ngrok_tunnel.public_url)


nest_asyncio.apply()

# finally run the app
uvicorn.run(app, port=port)

Public URL: https://8d75-109-184-39-15.ngrok-free.app


INFO:     Started server process [108280]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [108280]


KeyboardInterrupt: 

In [23]:
!pip install streamlit


  Using cached streamlit-1.35.0-py2.py3-none-any.whl (8.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 6.3 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 5.6 MB/s eta 0:00:00
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
